In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
df = pd.read_excel("./uni.xlsx", sheetname='MS Students')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560 entries, 0 to 559
Data columns (total 17 columns):
Year of Admission and Batch of the student    560 non-null int64
Gender                                        560 non-null object
Resedential Area and city of origin           560 non-null object
SSC Marks                                     1 non-null float64
HSSC Marks                                    0 non-null float64
Entry Test Score Marks                        539 non-null float64
1st Semester                                  529 non-null float64
2nd Semester                                  311 non-null float64
3rd Semester                                  266 non-null float64
4th Semester                                  137 non-null float64
5th Semester                                  114 non-null float64
6th Semester                                  25 non-null float64
7th Semester                                  14 non-null float64
8th Semester                      

/home/muhammadqasim/anaconda3/lib/python3.7/site-packages/pandas/io/excel.py:329: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  **kwds)


In [3]:
df.head()

,Year of Admission and Batch of the student,Gender,Resedential Area and city of origin,SSC Marks,HSSC Marks,Entry Test Score Marks,1st Semester,2nd Semester,3rd Semester,4th Semester,5th Semester,6th Semester,7th Semester,8th Semester,9th Semester,10th Semester,Cumulative GPA
0,2014,M,Karachi,NaN,NaN,39.92,3.33,2.71,0.00,3.67,3.33,1.67,0.0,NaN,NaN,NaN,2.86
1,2014,M,Karachi,NaN,NaN,26.58,2.00,1.78,2.50,1.78,2.67,0.00,0.0,2.0,0.0,0.0,2.64
2,2015,M,Karachi,NaN,NaN,0.00,3.44,2.86,2.67,3.17,4.00,0.00,NaN,NaN,NaN,NaN,3.21
3,2015,F,Karachi,NaN,NaN,0.00,2.22,3.14,2.34,2.67,2.33,0.00,0.0,0.0,NaN,NaN,2.54
4,2015,F,Karachi,NaN,NaN,0.00,3.38,3.55,3.11,2.33,0.00,0.00,NaN,NaN,NaN,NaN,3.24


In [4]:
[(x,len(df[df[x].isna()])) for x in df.columns]

[('Year of Admission and Batch of the student', 0),
 ('Gender', 0),
 ('Resedential Area and city of origin', 0),
 ('SSC Marks', 559),
 ('HSSC Marks', 560),
 ('Entry Test Score Marks', 21),
 ('1st Semester', 31),
 ('2nd Semester', 249),
 ('3rd Semester', 294),
 ('4th Semester', 423),
 ('5th Semester', 446),
 ('6th Semester', 535),
 ('7th Semester', 546),
 ('8th Semester', 554),
 ('9th Semester', 559),
 ('10th Semester', 559),
 ('Cumulative GPA', 0)]

# 1) What is the overall trend of our student's semester-wise performance over the past few years? Are we as faculty doing good with students?

In [5]:
len(df['Cumulative GPA'].unique())

146

In [6]:
df1=df[df['Cumulative GPA']<2.0].groupby(by=['Year of Admission and Batch of the student']).agg(['count'])[['Gender']]
df1.columns = ['CGPA < 2.0']
df1.head()

df2=df[df['Cumulative GPA']>=2.0].groupby(by=['Year of Admission and Batch of the student']).agg(['count'])[['Gender']]
df2.columns = ['CGPA >= 2.0']
df2.head()

,CGPA >= 2.0
Year of Admission and Batch of the student,
2014,2
2015,14
2016,19
2017,108
2018,145


In [7]:
df3=df.groupby(by=['Year of Admission and Batch of the student'], 
           sort=[False])[["Cumulative GPA"]].agg(['count','min','max','mean','std'])
df4 = pd.concat([df3,df1,df2], axis=1)

df4.columns = ['Total','Min','Max','Mean','STD','CGPA < 2.0','CGPA >= 2.0']

df4["PassingRatio"] = df4['CGPA >= 2.0'] / df4['Total'] * 100
df4['FailureRatio'] = 100 - df4.PassingRatio
df4

,Total,Min,Max,Mean,STD,CGPA < 2.0,CGPA >= 2.0,PassingRatio,FailureRatio
Year of Admission and Batch of the student,,,,,,,,,
2014,2,2.64,2.86,2.750000,0.155563,NaN,2,100.000000,0.000000
2015,14,2.23,3.24,2.740000,0.317587,NaN,14,100.000000,0.000000
2016,21,1.06,3.74,2.794286,0.631416,2.0,19,90.476190,9.523810
2017,114,0.00,3.98,2.816404,0.626924,6.0,108,94.736842,5.263158
2018,184,0.00,4.00,2.484674,1.222212,39.0,145,78.804348,21.195652
2019,225,0.00,4.00,0.017778,0.266667,224.0,1,0.444444,99.555556


In [8]:
q1=df.groupby(by=['Year of Admission and Batch of the student','Cumulative GPA'], 
           sort=[True, False])[["Gender"]].agg(['count'])

q1 = q1.reset_index()
q1.columns = ['Year','CGPA','NoStudentCounts']
q1.sort_values(by=['Year','NoStudentCounts'], ascending=[True,False])

,Year,CGPA,NoStudentCounts
0,2014,2.64,1
1,2014,2.86,1
2,2015,2.23,1
3,2015,2.36,1
4,2015,2.50,1
5,2015,2.53,1
6,2015,2.54,1
7,2015,2.58,1
8,2015,2.67,1
9,2015,2.69,1


In [9]:
df.columns

Index(['Year of Admission and Batch of the student', 'Gender',
       'Resedential Area and city of origin', 'SSC Marks', 'HSSC Marks',
       'Entry Test Score Marks', '1st Semester', '2nd Semester',
       '3rd Semester', '4th Semester', '5th Semester', '6th Semester',
       '7th Semester', '8th Semester', '9th Semester', '10th Semester',
       'Cumulative GPA'],
      dtype='object')

In [10]:
for year in df['Year of Admission and Batch of the student'].unique():
    print("===========",year,"=======")
    df1 = df[df['Year of Admission and Batch of the student']==year]
    print("Number of Students:",len(df1))
    df2 = pd.DataFrame(df1['Cumulative GPA'].value_counts())
    df2 = df2.reset_index()
    df2.columns=['CGPA','count']
    print(df2)

=========== 2014 =======
Number of Students: 2
   CGPA  count
0  2.64      1
1  2.86      1
=========== 2015 =======
Number of Students: 14
    CGPA  count
0   2.53      1
1   3.13      1
2   2.23      1
3   2.58      1
4   3.24      1
5   2.69      1
6   3.01      1
7   2.67      1
8   2.54      1
9   2.93      1
10  3.21      1
11  2.74      1
12  2.36      1
13  2.50      1
=========== 2016 =======
Number of Students: 21
    CGPA  count
0   2.91      1
1   2.15      1
2   2.57      1
3   2.61      1
4   3.59      1
5   2.72      1
6   2.93      1
7   3.08      1
8   3.29      1
9   3.17      1
10  2.81      1
11  3.06      1
12  3.74      1
13  2.65      1
14  1.92      1
15  3.67      1
16  2.52      1
17  3.43      1
18  2.30      1
19  1.06      1
20  2.50      1
=========== 2017 =======
Number of Students: 114
    CGPA  count
0   3.00      4
1   2.92      4
2   2.77      4
3   3.09      3
4   3.33      3
5   2.33      3
6   2.89      3
7   2.41      2
8   2.85      2
9   2.70   